In [168]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [169]:
sql = "SELECT * \
from ds_qf60"
print(sql)
df = pd.read_sql_query(sql, conn, index_col=None)

df.shape

SELECT * from ds_qf60


(35170, 46)

In [170]:
# df.info()

In [171]:
col_list_ds = ['symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'sz_jt_10','class0' ]
df.tail(1)[col_list_ds]

,symbol,trade_date,pct_change,diff,dea,bar,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_5_20,pct_fl_20_60,sz_max_5,sz_max_10,sz_max_20,sz_jt_10,class0
35169,689009,2021-04-19,3.04,-0.461137,-1.14476,1.36724,39,0.0,-0.143429,0.522034,...,0,1.30432,1.13952,0.745176,0.524615,0.379001,4.43571,4.43571,0,0


In [172]:
df['sz_jt_10'].value_counts()

0    22561
1     8740
2     3869
Name: sz_jt_10, dtype: int64

In [173]:
# 开发测试：条件取数
# sum = len(df)

# cond2a = df['sz_jt_10']>=2
# df[ cond2a ]['sz_jt_10'].value_counts()

# cond2b = df['sz_jt_10']>1 
# cond2c = df['sz_jt_10']<=2
# cond2bc2 = ( cond2b & cond2c )
# df[ cond2bc2 ]['sz_jt_10'].value_counts()

In [174]:
# jx_dg_5 预处理
min_dg_5, max_dg_5 = df['jx_dg_5'].min(), df['jx_dg_5'].max()
step_dg_5 = (max_dg_5 - min_dg_5)/steps
# from_dg_5 = min_dg_5
# min_dg_5, max_dg_5, step_dg_5, from_dg_5  

In [175]:
# fl_1_3 处理
min_fl_1_3, max_fl_1_3 = df['pct_fl_1_3'].min(), df['pct_fl_1_3'].max()
step_fl_1_3 = (max_fl_1_3 - min_fl_1_3)/steps
from_fl_1_3 = min_fl_1_3
min_fl_1_3, max_fl_1_3, from_fl_1_3, step_fl_1_3 

(1.15793, 66.3237, 1.15793, 13.033154000000001)

In [176]:
# # df['sz_jt_10'].value_counts()
# test1 = df[df['sz_jt_10'] >=3]['sz_jt_10'].value_counts()
# # len( df[df['sz_jt_10'] >=1]['sz_jt_10'].value_counts() )
# test1, type(test1)
# len(test1)
# num = 1 if test1.empty else 0

# lambda x
# test1.iloc[0] + test1.iloc[1]

In [188]:
%%time
# 初始化数据
# col = ['pct_change',
#     'diff', 'dea',\
#     'jx_days_ud60',\
#     'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
#     'jx_zs_5', 'jx_zs_10', 'jx_zs_20', \
#     'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
#     'sz_max_10' ]
col = ['jx_dg_5','pct_fl_1_3','step', 'class', 'rate' ]
# man[col] = 0.0
man_data = {'jx_dg_5':0.0,
            'pct_fl_1_3':0.0,
            'step':0, 'class':0, 'rate':0.0    
}
man = pd.DataFrame(data=man_data, index=[0])
 
# man.drop(man.index, inplace=True)
# man.set_index('rate', drop=False)

# 开始epoch
steps = 5
cond_begin = df['sz_jt_10'] >=2

from_dg_5 = min_dg_5
for i_dg_5 in range(steps):
    to_dg_5 = min_dg_5 + (i_dg_5+1)*step_dg_5
    # 处理条件
    cond_dg_5 = (df['jx_dg_5'] >= from_dg_5) & (df['jx_dg_5'] < to_dg_5 )
    # cond_fl_1_3_class = ( cond_fl_1_3 ) & (cond_last)
    cond_last = (cond_dg_5) & ( cond_begin )
    
    from_fl_1_3 = min_fl_1_3
    for i_fl_1_3 in range(steps):
        to_fl_1_3 = min_fl_1_3 + (i_fl_1_3+1)*step_fl_1_3
        # print('%.0f : , %.4f,  %.4f'%(i, from_fl_1_3, to_fl_1_3), end=' ' )
        # 处理条件
        cond_fl_1_3 = (df['pct_fl_1_3'] >= from_fl_1_3) & (df['pct_fl_1_3'] < to_fl_1_3 )
        cond_fl_1_3_class = ( cond_fl_1_3 ) & (cond_last)
        
        # 开始计算   
        sum_step_series  = df[ cond_fl_1_3 ]['sz_jt_10'].value_counts() 
        sum_class_series = df[ cond_fl_1_3_class ]['sz_jt_10'].value_counts() 
        # 
        sum_step = sum_step_series.iloc[0] if not sum_step_series.empty else 0
        sum_class = sum_class_series.iloc[0] if not sum_class_series.empty else 0
        sum_class_rate = 0 if sum_class==0 else sum_class/sum_step
        # print( i_dg_5, i_fl_1_3, ':  %.0f %.0f %.4f' %(sum_step, sum_class, sum_class_rate) )
        # 记录到man
        # new_data = {'rate': sum_class_rate,'jx_dg_5': from_dg_5, 'pct_fl_1_3': from_fl_1_3}
        man_list = [from_dg_5, from_fl_1_3, sum_step, sum_class, sum_class_rate]
        man = pd.DataFrame(np.insert(man.values, len(man.index), values=man_list, axis=0))
        
        # man = man.append(new_data, ignore_index=True)
        # new_series = pd.Series( new_data)
        # man = pd.concat([man, new_series], axis=0, ignore_index=True)
        # man = pd.concat([man, new_series], axis=1)
        # man = pd.concat(man, new_series )
        # 下一个loop
        from_fl_1_3 += step_fl_1_3
    # jx_dg_5下一个loop
    from_dg_5 += step_dg_5
man.shape

CPU times: total: 46.9 ms
Wall time: 99.9 ms


(26, 5)

In [201]:
# man.set_index(['rate'], drop=False)
pd.set_option('display.max_rows', None)

# col = ['jx_dg_5','pct_fl_1_3','step', 'class', 'rate' ]
man.columns = col
# man.sort_index(ascending=True)
man.sort_values('rate', ascending=False)
man.set_index('rate', drop=False)
sorting = man.sort_values('rate', ascending=False)
sorting

,jx_dg_5,pct_fl_1_3,step,class,rate
7,-0.243178,14.191084,2.0,1.0,0.500000
6,-0.243178,1.157930,22558.0,3462.0,0.153471
11,1.370374,1.157930,22558.0,287.0,0.012723
1,-1.856730,1.157930,22558.0,111.0,0.004921
16,2.983926,1.157930,22558.0,6.0,0.000266
21,4.597478,1.157930,22558.0,1.0,0.000044
0,0.000000,0.000000,0.0,0.0,0.000000
15,1.370374,53.290546,1.0,0.0,0.000000
24,4.597478,40.257392,0.0,0.0,0.000000
23,4.597478,27.224238,1.0,0.0,0.000000


,jx_dg_5,pct_fl_1_3,step,class,rate
7,-0.243178,14.191084,2.0,1.0,0.500000
6,-0.243178,1.157930,22558.0,3462.0,0.153471
11,1.370374,1.157930,22558.0,287.0,0.012723
1,-1.856730,1.157930,22558.0,111.0,0.004921
16,2.983926,1.157930,22558.0,6.0,0.000266
21,4.597478,1.157930,22558.0,1.0,0.000044
0,0.000000,0.000000,0.0,0.0,0.000000
15,1.370374,53.290546,1.0,0.0,0.000000
24,4.597478,40.257392,0.0,0.0,0.000000
23,4.597478,27.224238,1.0,0.0,0.000000


In [195]:
man

,jx_dg_5,pct_fl_1_3,step,class,rate
0,0.000000,0.000000,0.0,0.0,0.000000
1,-1.856730,1.157930,22558.0,111.0,0.004921
2,-1.856730,14.191084,2.0,0.0,0.000000
3,-1.856730,27.224238,1.0,0.0,0.000000
4,-1.856730,40.257392,0.0,0.0,0.000000
5,-1.856730,53.290546,1.0,0.0,0.000000
6,-0.243178,1.157930,22558.0,3462.0,0.153471
7,-0.243178,14.191084,2.0,1.0,0.500000
8,-0.243178,27.224238,1.0,0.0,0.000000
9,-0.243178,40.257392,0.0,0.0,0.000000
